In [35]:
import os
import tensorflow as tf
import tensorflow.keras.backend as K
import math
import numpy as np
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D
from tensorflow.keras.metrics import Metric
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math


# Tái định nghĩa ArcFaceLayer (cần thiết để load model)
class ArcFaceLayer(tf.keras.layers.Layer):
    def __init__(self, num_classes, margin=0.5, scale=30.0, embedding_size=512, **kwargs):
        super(ArcFaceLayer, self).__init__(**kwargs)
        self.num_classes = num_classes
        self.margin = margin
        self.scale = scale
        self.embedding_size = embedding_size
        self.cos_m = tf.cos(margin)
        self.sin_m = tf.sin(margin)
        self.threshold = tf.cos(math.pi - margin)
        self.mm = tf.sin(math.pi - margin) * margin

    def build(self, input_shape):
        self.W = self.add_weight(
            name='W',
            shape=(self.embedding_size, self.num_classes),
            initializer='glorot_uniform',
            trainable=True
        )
        super(ArcFaceLayer, self).build(input_shape)

    def call(self, inputs, training=None):
        embeddings, labels = inputs
        # Normalize embeddings and weights
        embeddings = tf.nn.l2_normalize(embeddings, axis=1)
        W_norm = tf.nn.l2_normalize(self.W, axis=0)

        # Compute cosine similarity
        cos_theta = tf.matmul(embeddings, W_norm)
        cos_theta = tf.clip_by_value(cos_theta, -1.0 + K.epsilon(), 1.0 - K.epsilon())

        # Compute sine
        sin_theta = tf.sqrt(1.0 - tf.square(cos_theta))

        # Compute cos(theta + margin)
        cos_theta_m = cos_theta * self.cos_m - sin_theta * self.sin_m

        # Apply threshold
        cos_theta_m = tf.where(cos_theta > self.threshold, cos_theta_m, cos_theta - self.mm)

        one_hot = tf.cast(labels, dtype=cos_theta.dtype)
        logits = (one_hot * cos_theta_m) + ((1.0 - one_hot) * cos_theta)
        logits = logits * self.scale

        return logits

    def get_config(self):
        config = super().get_config()
        config.update({
            'num_classes': self.num_classes,
            'margin': self.margin,
            'scale': self.scale,
            'embedding_size': self.embedding_size
        })
        return config

# Tái định nghĩa ArcFaceAccuracy (cần thiết để load model)
class ArcFaceAccuracy(Metric):
    def __init__(self, name='arcface_accuracy', **kwargs):
        super(ArcFaceAccuracy, self).__init__(name=name, **kwargs)
        self.total = self.add_weight(name='total', initializer='zeros')
        self.count = self.add_weight(name='count', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        predicted_labels = tf.argmax(y_pred, axis=1)
        true_labels = tf.argmax(y_true, axis=1)
        matches = tf.cast(tf.equal(predicted_labels, true_labels), tf.float32)

        if sample_weight is not None:
            sample_weight = tf.cast(sample_weight, tf.float32)
            matches = matches * sample_weight

        self.total.assign_add(tf.reduce_sum(matches))
        self.count.assign_add(tf.cast(tf.size(matches), tf.float32))

    def result(self):
        return tf.math.divide_no_nan(self.total, self.count)

    def reset_states(self):
        self.total.assign(0.0)
        self.count.assign(0.0)

    def get_config(self):
        return {'name': self.name}

# Hàm mất mát ArcFace
def arcface_loss(y_true, y_pred):
    return tf.keras.losses.categorical_crossentropy(y_true, y_pred, from_logits=True)

class ArcFaceModelLoader:
    def __init__(self):
        self.full_model = None
        self.embedding_model = None

    def load_full_model(self, model_path):
        """
        Load mô hình ArcFace đầy đủ (bao gồm classification head)
        """
        try:
            # Đăng ký custom objects
            custom_objects = {
                'ArcFaceLayer': ArcFaceLayer,
                'ArcFaceAccuracy': ArcFaceAccuracy,
                'arcface_loss': arcface_loss
            }

            self.full_model = load_model(model_path, custom_objects=custom_objects)
            print(f"✓ Đã load thành công mô hình đầy đủ từ: {model_path}")
            print(f"Model inputs: {[inp.name for inp in self.full_model.inputs]}")
            print(f"Model output shape: {self.full_model.output.shape}")
            return self.full_model

        except Exception as e:
            print(f"✗ Lỗi khi load mô hình đầy đủ: {e}")
            return None

    def load_embedding_model(self, model_path):
        """
        Load mô hình chỉ để extract embeddings
        """
        try:
            self.embedding_model = load_model(model_path)
            print(f"✓ Đã load thành công mô hình embedding từ: {model_path}")
            print(f"Model input shape: {self.embedding_model.input.shape}")
            print(f"Model output shape: {self.embedding_model.output.shape}")
            return self.embedding_model

        except Exception as e:
            print(f"✗ Lỗi khi load mô hình embedding: {e}")
            return None

    def preprocess_image(self, image_path, target_size=(112, 112)):
        """
        Tiền xử lý ảnh để đưa vào model
        """
        try:
            # Đọc và decode ảnh
            image = tf.io.read_file(image_path)
            image = tf.image.decode_jpeg(image, channels=3)
            image = tf.image.convert_image_dtype(image, tf.float32)

            # Resize về kích thước mong muốn
            image = tf.image.resize(image, target_size)

            # Chuẩn hóa về [-1, 1]
            image = (image - 0.5) * 2.0

            # Thêm batch dimension
            image = tf.expand_dims(image, 0)

            return image

        except Exception as e:
            print(f"✗ Lỗi khi xử lý ảnh {image_path}: {e}")
            return None

    def extract_embedding(self, image_input):
        """
        Extract embedding từ ảnh
        """
        if self.embedding_model is None:
            print("✗ Chưa load mô hình embedding!")
            return None

        try:
            # Nếu input là đường dẫn file
            if isinstance(image_input, str):
                image = self.preprocess_image(image_input)
                if image is None:
                    return None
            else:
                # Nếu input đã là tensor
                image = image_input

            # Extract embedding
            embedding = self.embedding_model.predict(image, verbose=0)

            # Normalize embedding
            embedding = tf.nn.l2_normalize(embedding, axis=1)

            return embedding.numpy()

        except Exception as e:
            print(f"✗ Lỗi khi extract embedding: {e}")
            return None

    def predict_identity(self, image_input, dummy_labels=None, num_classes=None):
        """
        Dự đoán danh tính từ ảnh (sử dụng full model)
        """
        if self.full_model is None:
            print("✗ Chưa load mô hình đầy đủ!")
            return None

        try:
            # Nếu input là đường dẫn file
            if isinstance(image_input, str):
                image = self.preprocess_image(image_input)
                if image is None:
                    return None
            else:
                # Nếu input đã là tensor
                image = image_input

            # Tạo dummy labels nếu chưa có (cần thiết cho ArcFace layer)
            if dummy_labels is None:
                if num_classes is None:
                    # Cố gắng lấy num_classes từ model
                    num_classes = self.full_model.output.shape[-1]
                dummy_labels = np.zeros((image.shape[0], num_classes))

            # Dự đoán
            predictions = self.full_model.predict([image, dummy_labels], verbose=0)

            # Lấy class có xác suất cao nhất
            predicted_class = np.argmax(predictions, axis=1)
            confidence = np.max(tf.nn.softmax(predictions), axis=1)

            return predicted_class, confidence.numpy(), predictions

        except Exception as e:
            print(f"✗ Lỗi khi dự đoán: {e}")
            return None

    def compute_similarity(self, embedding1, embedding2):
        """
        Tính độ tương đồng cosine giữa 2 embeddings
        """
        try:
            # Ensure embeddings are normalized
            emb1_norm = tf.nn.l2_normalize(embedding1, axis=1)
            emb2_norm = tf.nn.l2_normalize(embedding2, axis=1)

            # Compute cosine similarity
            similarity = tf.reduce_sum(emb1_norm * emb2_norm, axis=1)

            return similarity.numpy()

        except Exception as e:
            print(f"✗ Lỗi khi tính similarity: {e}")
            return None

# Ví dụ sử dụng

# Khởi tạo loader
loader = ArcFaceModelLoader()

# Đường dẫn đến các model đã train
embedding_model_path = r'C:\Users\admin\OneDrive - Hanoi University of Science and Technology\Documents\GitHub\PTTK\face_recognization\model\arcface_embedding_model.h5'

# Load models
print("=== LOADING MODELS ===")
embedding_model = loader.load_embedding_model(embedding_model_path)

# Test với một ảnh (thay đổi đường dẫn theo ảnh thực tế của bạn)
test_image_path = '/kaggle/input/face-align/aligned_faces/person1_001.jpg'  # Thay đổi đường dẫn

if os.path.exists(test_image_path):
    print(f"\n=== TESTING WITH IMAGE: {test_image_path} ===")

    # 1. Extract embedding
    print("\n1. Extracting embedding...")
    embedding = loader.extract_embedding(test_image_path)
    if embedding is not None:
        print(f"✓ Embedding shape: {embedding.shape}")
        print(f"✓ Embedding norm: {np.linalg.norm(embedding):.4f}")

    # 2. Predict identity (nếu có full model)


    # 3. Test similarity (so sánh ảnh với chính nó)
    print("\n3. Testing similarity...")
    embedding1 = loader.extract_embedding(test_image_path)
    embedding2 = loader.extract_embedding(test_image_path)  # Cùng ảnh

    if embedding1 is not None and embedding2 is not None:
        similarity = loader.compute_similarity(embedding1, embedding2)
        print(f"✓ Self-similarity: {similarity[0]:.4f} (should be ~1.0)")

else:
    print(f"✗ Không tìm thấy ảnh test: {test_image_path}")
    print("Vui lòng thay đổi đường dẫn test_image_path trong code")


=== LOADING MODELS ===


✓ Đã load thành công mô hình embedding từ: C:\Users\admin\OneDrive - Hanoi University of Science and Technology\Documents\GitHub\PTTK\face_recognization\model\arcface_embedding_model.h5
Model input shape: (None, 112, 112, 3)
Model output shape: (None, 512)
✗ Không tìm thấy ảnh test: /kaggle/input/face-align/aligned_faces/person1_001.jpg
Vui lòng thay đổi đường dẫn test_image_path trong code


In [36]:

import cv2
import numpy as np
import os
from PIL import Image
from facenet_pytorch import MTCNN
import torch

# Template landmarks cho căn chỉnh khuôn mặt
TEMPLATE = np.array([
    [38.2946, 51.6963],  # left eye
    [73.5318, 51.5014],  # right eye
    [56.0252, 71.7366],  # nose
    [41.5493, 92.3655],  # left mouth
    [70.7299, 92.2041],  # right mouth
], dtype=np.float32)

class FaceLoading:
    def __init__(self, directory):
        self.directory = directory
        self.target_size = (112, 112)
        self.X = []
        self.y = []
        self.mtcnn = MTCNN(image_size=160, margin=0, min_face_size=20,
                           thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
                           device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu'))

    def extract_face(self, path):
        """
        Trích xuất và căn chỉnh khuôn mặt từ ảnh
        """
        try:
            # Đọc ảnh
            img = cv2.imread(path)
            if img is None:
                print(f"Cannot read image: {path}")
                return None

            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img_pil = Image.fromarray(img)

            # Phát hiện khuôn mặt và landmarks
            boxes, probs, landmarks = self.mtcnn.detect(img_pil, landmarks=True)

            if boxes is None or len(boxes) == 0:
                print(f"No face detected in: {path}")
                return None

            # Căn chỉnh khuôn mặt nếu có landmarks
            if landmarks is not None and len(landmarks) > 0:
                landmark = landmarks[0]
                if landmark is not None and len(landmark) == 5:
                    # Căn chỉnh khuôn mặt
                    src = np.array(landmark).astype(np.float32)
                    M = cv2.estimateAffinePartial2D(src, TEMPLATE, method=cv2.LMEDS)[0]

                    if M is not None:
                        aligned_face = cv2.warpAffine(img, M, self.target_size, borderValue=0.0)
                        return aligned_face

            # Fallback: sử dụng phương pháp cũ nếu không căn chỉnh được
            face_tensor = self.mtcnn(img_pil)
            if face_tensor is None:
                print(f"Cannot extract face from: {path}")
                return None

            face = face_tensor.permute(1, 2, 0).cpu().numpy()
            face = cv2.resize(face, self.target_size)
            # Chuyển đổi từ [-1, 1] về [0, 255]
            face = ((face + 1) * 127.5).astype(np.uint8)
            return face

        except Exception as e:
            print(f"Error processing {path}: {e}")
            return None

    def load_face_and_class(self):
        """
        Tải tất cả khuôn mặt và nhãn từ thư mục
        """
        print(f"Loading faces from: {self.directory}")

        for sub_dir in os.listdir(self.directory):
            sub_dir_path = os.path.join(self.directory, sub_dir)

            if not os.path.isdir(sub_dir_path):
                continue

            print(f"Processing class: {sub_dir}")
            face_count = 0

            for img_name in os.listdir(sub_dir_path):
                # Chỉ xử lý file ảnh
                if not img_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
                    continue

                full_path = os.path.join(sub_dir_path, img_name)
                face = self.extract_face(full_path)

                if face is not None:
                    self.X.append(face)
                    self.y.append(sub_dir)
                    face_count += 1

            print(f"Loaded {face_count} faces for class: {sub_dir}")

        print(f"Total faces loaded: {len(self.X)}")
        return np.array(self.X), np.array(self.y)

    def get_class_counts(self):
        """
        Trả về số lượng mẫu cho mỗi class
        """
        unique, counts = np.unique(self.y, return_counts=True)
        return dict(zip(unique, counts))

    def normalize_faces(self, X):
        """
        Chuẩn hóa dữ liệu khuôn mặt về [-1, 1]
        """
        return (X.astype(np.float32) - 127.5) / 128.0
face_loading = FaceLoading(r'C:\Users\admin\OneDrive - Hanoi University of Science and Technology\Documents\GitHub\PTTK\face_recognization\source\data_raw\image')
X,y = face_loading.load_face_and_class()

Loading faces from: C:\Users\admin\OneDrive - Hanoi University of Science and Technology\Documents\GitHub\PTTK\face_recognization\source\data_raw\image
Processing class: 1
Loaded 13 faces for class: 1
Processing class: 2
Loaded 6 faces for class: 2
Processing class: 3
Loaded 8 faces for class: 3
Processing class: 4
Loaded 19 faces for class: 4
Processing class: 5
Loaded 20 faces for class: 5
Processing class: 6
Loaded 21 faces for class: 6
Total faces loaded: 87


In [38]:
def l2_normalize(x, axis=-1, epsilon=1e-10):
    return x / np.sqrt(np.maximum(np.sum(np.square(x), axis=axis, keepdims=True), epsilon))

In [39]:
embeddings_data = []
labels_data = []
for face,labels in zip(X,y):
    embed = loader.extract_embedding(np.expand_dims(face, axis=0))
    embeddings_data.append(embed)
    labels_data.append(labels)
np.save("embeddings_data", np.array(embeddings_data))
np.save("labels_data", np.array(labels_data))
print("Lưu embedding data thành công")

Lưu embedding data thành công


In [40]:
embeddings_data = np.load("embeddings_data.npy")
labels_data = np.load("labels_data.npy")
face = []

In [41]:
def predict(img_path, loader: ArcFaceModelLoader, embed_data, label_data, threshold=0.1):

    embed_img = loader.extract_embedding(img_path)
    if embed_img is None:
        return "Unknown", 0

    best_similarity = -1
    best_label = "Unknown"

    for emb, label in zip(embed_data, label_data):
        similarity = loader.compute_similarity(embed_img, emb)
        if similarity > best_similarity:
            best_similarity = similarity
            best_label = label

    if best_similarity < threshold:
        return "Unknown", float(best_similarity)

    return best_label, float(best_similarity)



In [42]:
print(predict(r'C:\Users\admin\OneDrive - Hanoi University of Science and Technology\Documents\GitHub\PTTK\face_recognization\source\data_raw\image\1\z6765814472987_869536c9a66d94a5f7554f8d4d94999c.jpg',loader, embeddings_data, labels_data))

('Unknown', -0.015392984263598919)


C:\Users\admin\AppData\Local\Temp\ipykernel_20332\333147092.py:17: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return "Unknown", float(best_similarity)
